# Homework and bake-off: Relation extraction using distant supervision

In [1]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2020"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Baselines](#Baselines)
  1. [Hand-build feature functions](#Hand-build-feature-functions)
  1. [Distributed representations](#Distributed-representations)
1. [Homework questions](#Homework-questions)
  1. [Different model factory [1 points]](#Different-model-factory-[1-points])
  1. [Directional unigram features [1.5 points]](#Directional-unigram-features-[1.5-points])
  1. [The part-of-speech tags of the "middle" words [1.5 points]](#The-part-of-speech-tags-of-the-"middle"-words-[1.5-points])
  1. [Bag of Synsets [2 points]](#Bag-of-Synsets-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bake-off [1 point]](#Bake-off-[1-point])

## Overview

This homework and associated bake-off are devoted to developing really effective relation extraction systems using distant supervision. 

As with the previous assignments, this notebook first establishes a baseline system. The initial homework questions ask you to create additional baselines and suggest areas for innovation, and the final homework question asks you to develop an original system for you to enter into the bake-off.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [2]:
import numpy as np
import os
import rel_ext
from sklearn.linear_model import LogisticRegression
import utils

As usual, we unite our corpus and KB into a dataset, and create some splits for experimentation:

In [3]:
rel_ext_data_home = os.path.join('data', 'rel_ext_data')

In [4]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'corpus.tsv.gz'))

In [5]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'kb.tsv.gz'))

In [6]:
dataset = rel_ext.Dataset(corpus, kb)

You are not wedded to this set-up for splits. The bake-off will be conducted on a previously unseen test-set, so all of the data in `dataset` is fair game:

In [7]:
splits = dataset.build_splits(
    split_names=['tiny', 'train', 'dev'],
    split_fracs=[0.01, 0.79, 0.20],
    seed=1)

In [8]:
splits

{'tiny': Corpus with 3,474 examples; KB with 445 triples,
 'train': Corpus with 263,285 examples; KB with 36,191 triples,
 'dev': Corpus with 64,937 examples; KB with 9,248 triples,
 'all': Corpus with 331,696 examples; KB with 45,884 triples}

## Baselines

### Hand-build feature functions

In [9]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

In [10]:
featurizers = [simple_bag_of_words_featurizer]

In [11]:
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')

In [12]:
baseline_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.856      0.368      0.676        340       5716
author                    0.812      0.536      0.737        509       5885
capital                   0.640      0.168      0.410         95       5471
contains                  0.800      0.592      0.747       3904       9280
film_performance          0.760      0.572      0.713        766       6142
founders                  0.818      0.403      0.678        380       5756
genre                     0.600      0.176      0.405        170       5546
has_sibling               0.851      0.240      0.564        499       5875
has_spouse                0.894      0.313      0.652        594       5970
is_a                      0.728      0.221      0.500        497       5873
nationality               0.625      0.199      0.438        301       5677
parents     

Studying model weights might yield insights:

In [13]:
rel_ext.examine_model_weights(baseline_results)

Highest and lowest feature weights for relation adjoins:

     2.551 Córdoba
     2.518 Taluks
     2.324 Valais
     ..... .....
    -1.270 towers
    -1.511 who
    -1.524 he

Highest and lowest feature weights for relation author:

     2.866 wrote
     2.416 writer
     2.315 book
     ..... .....
    -2.636 Alice
    -3.360 1945
    -6.998 1865

Highest and lowest feature weights for relation capital:

     3.057 capital
     1.783 km
     1.754 especially
     ..... .....
    -1.898 ~3.9
    -1.909 million
    -1.915 pop

Highest and lowest feature weights for relation contains:

     2.501 bordered
     2.371 tiny
     2.165 surrounding
     ..... .....
    -2.423 who
    -2.460 film
    -8.242 Bronx

Highest and lowest feature weights for relation film_performance:

     4.170 starring
     3.914 co-starring
     3.758 alongside
     ..... .....
    -1.959 Malice
    -1.967 Wonderland
    -2.141 comedian

Highest and lowest feature weights for relation founders:

     3.938 fou

### Distributed representations

This simple baseline sums the GloVe vector representations for all of the words in the "middle" span and feeds those representations into the standard `LogisticRegression`-based `model_factory`. The crucial parameter that enables this is `vectorize=False`. This essentially says to `rel_ext.experiment` that your featurizer or your model will do the work of turning examples into vectors; in that case, `rel_ext.experiment` just organizes these representations by relation type.

In [14]:
GLOVE_HOME = os.path.join('data', 'glove.6B')

In [15]:
glove_lookup = utils.glove2dict(
    os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

In [16]:
def glove_middle_featurizer(kbt, corpus, np_func=np.sum):
    reps = []
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split():
            rep = glove_lookup.get(word)
            if rep is not None:
                reps.append(rep)
    # A random representation of the right dimensionality if the
    # example happens not to overlap with GloVe's vocabulary:
    if len(reps) == 0:
        dim = len(next(iter(glove_lookup.values())))                
        return utils.randvec(n=dim)
    else:
        return np_func(reps, axis=0)

In [17]:
glove_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=[glove_middle_featurizer],    
    vectorize=False, # Crucial for this featurizer!
    verbose=True)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.870      0.432      0.723        340       5716
author                    0.837      0.444      0.711        509       5885
capital                   0.615      0.168      0.402         95       5471
contains                  0.656      0.405      0.583       3904       9280
film_performance          0.788      0.321      0.611        766       6142
founders                  0.795      0.234      0.537        380       5756
genre                     0.476      0.059      0.197        170       5546
has_sibling               0.821      0.238      0.551        499       5875
has_spouse                0.876      0.355      0.677        594       5970
is_a                      0.664      0.147      0.390        497       5873
nationality               0.642      0.233      0.475        301       5677
parents     

With the same basic code design, one can also use the PyTorch models included in the course repo, or write new ones that are better aligned with the task. For those models, it's likely that the featurizer will just return a list of tokens (or perhaps a list of lists of tokens), and the model will map those into vectors using an embedding.

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Different model factory [1 points]

The code in `rel_ext` makes it very easy to experiment with other classifier models: one need only redefine the `model_factory` argument. This question asks you to assess a [Support Vector Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html).

__To submit:__ A wrapper function `run_svm_model_factory` that does the following: 

1. Uses `rel_ext.experiment` with the model factory set to one based in an `SVC` with `kernel='linear'` and all other arguments left with default values. 
1. Trains on the 'train' part of `splits`.
1. Assesses on the `dev` part of `splits`.
1. Uses `featurizers` as defined above. 
1. Returns the return value of `rel_ext.experiment` for this set-up.

The function `test_run_svm_model_factory` will check that your function conforms to these general specifications.

In [18]:
def run_svm_model_factory():
    
    ##### YOUR CODE HERE
    from sklearn.svm import SVC
    glove_results = rel_ext.experiment(
        splits,
        train_split='train',
        test_split='dev',
        model_factory=(lambda: SVC(kernel='linear', max_iter=2)),
        featurizers=[glove_middle_featurizer],
        vectorize=False,  # Crucial for this featurizer!
        verbose=True)

    return glove_results

In [19]:
def test_run_svm_model_factory(run_svm_model_factory):
    results = run_svm_model_factory()
    assert 'featurizers' in results, \
        "The return value of `run_svm_model_factory` seems not to be correct"
    # Check one of the models to make sure it's an SVC:
    assert 'SVC' in results['models']['adjoins'].__class__.__name__, \
        "It looks like the model factor wasn't set to use an SVC."    

In [20]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_svm_model_factory(run_svm_model_factory)

C:\Users\agoswami\AppData\Local\Continuum\anaconda3\envs\nlu\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=2).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\agoswami\AppData\Local\Continuum\anaconda3\envs\nlu\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=2).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\agoswami\AppData\Local\Continuum\anaconda3\envs\nlu\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=2).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\agoswami\AppData\Local\Continuum\anaconda3\envs\nlu\lib\site-packages\sklearn\svm\_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=2).  Consider pre-p

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.054      0.897      0.066        340       5716
author                    0.086      0.994      0.105        509       5885
capital                   0.017      0.989      0.021         95       5471
contains                  0.421      0.999      0.476       3904       9280
film_performance          0.125      0.997      0.151        766       6142
founders                  0.066      1.000      0.081        380       5756
genre                     0.032      0.806      0.039        170       5546
has_sibling               0.083      0.956      0.102        499       5875
has_spouse                0.099      0.997      0.121        594       5970
is_a                      0.084      0.996      0.103        497       5873
nationality               0.053      1.000      0.065        301       5677
parents     

### Directional unigram features [1.5 points]

The current bag-of-words representation makes no distinction between "forward" and "reverse" examples. But, intuitively, there is big difference between _X and his son Y_ and _Y and his son X_. This question asks you to modify `simple_bag_of_words_featurizer` to capture these differences. 

__To submit:__

1. A feature function `directional_bag_of_words_featurizer` that is just like `simple_bag_of_words_featurizer` except that it distinguishes "forward" and "reverse". To do this, you just need to mark each word feature for whether it is derived from a subject–object example or from an object–subject example.  The included function `test_directional_bag_of_words_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `directional_bag_of_words_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

3. `rel_ext.experiment` returns some of the core objects used in the experiment. How many feature names does the `vectorizer` have for the experiment run in the previous step? Include the code needed for getting this value. (Note: we're partly asking you to figure out how to get this value by using the sklearn documentation, so please don't ask how to do it!)

In [21]:
def directional_bag_of_words_featurizer(kbt, corpus, feature_counter): 
    # Append these to the end of the keys you add/access in 
    # `feature_counter` to distinguish the two orders. You'll
    # need to use exactly these strings in order to pass 
    # `test_directional_bag_of_words_featurizer`.
    subject_object_suffix = "_SO"
    object_subject_suffix = "_OS"
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word + subject_object_suffix] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word + object_subject_suffix] += 1

    return feature_counter


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
featurizers = [directional_bag_of_words_featurizer]
model_factory = lambda: LogisticRegression(fit_intercept=True, solver='liblinear')
directional_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)

len(directional_results['vectorizer'].get_feature_names())

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.876      0.394      0.704        340       5716
author                    0.844      0.583      0.775        509       5885
capital                   0.778      0.221      0.517         95       5471
contains                  0.833      0.639      0.785       3904       9280
film_performance          0.835      0.655      0.792        766       6142
founders                  0.807      0.408      0.675        380       5756
genre                     0.712      0.276      0.541        170       5546
has_sibling               0.838      0.248      0.568        499       5875
has_spouse                0.908      0.348      0.687        594       5970
is_a                      0.740      0.252      0.533        497       5873
nationality               0.651      0.236      0.482        301       5677
parents     

40666

In [22]:
def test_directional_bag_of_words_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['is_OS'] += 5
    feature_counter = directional_bag_of_words_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'is_OS':6,'a_OS':1,'webcomic_OS':1,'created_OS':1,'by_OS':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [23]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_directional_bag_of_words_featurizer(corpus)

### The part-of-speech tags of the "middle" words [1.5 points]

Our corpus distribution contains part-of-speech (POS) tagged versions of the core text spans. Let's begin to explore whether there is information in these sequences, focusing on `middle_POS`.

__To submit:__

1. A feature function `middle_bigram_pos_tag_featurizer` that is just like `simple_bag_of_words_featurizer` except that it creates a feature for bigram POS sequences. For example, given 

  `The/DT dog/N napped/V`
  
   we obtain the list of bigram POS sequences
  
   `b = ['<s> DT', 'DT N', 'N V', 'V </s>']`. 
   
   Of course, `middle_bigram_pos_tag_featurizer` should return count dictionaries defined in terms of such bigram POS lists, on the model of `simple_bag_of_words_featurizer`.  Don't forget the start and end tags, to model those environments properly! The included function `test_middle_bigram_pos_tag_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `middle_bigram_pos_tag_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment` as exemplified above in this notebook.)

In [24]:
def middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for tag_bigram in get_tag_bigrams(ex.middle_POS):
            feature_counter[tag_bigram] += 1

    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for tag_bigram in get_tag_bigrams(ex.middle_POS):
            feature_counter[tag_bigram] += 1

    return feature_counter


def get_tag_bigrams(s):
    """Suggested helper method for `middle_bigram_pos_tag_featurizer`.
    This should be defined so that it returns a list of str, where each 
    element is a POS bigram."""
    # The values of `start_symbol` and `end_symbol` are defined
    # here so that you can use `test_middle_bigram_pos_tag_featurizer`.
    start_symbol = "<s>"
    end_symbol = "</s>"
    
    ##### YOUR CODE HERE
    tags = [start_symbol] + get_tags(s) + [end_symbol]
    tag_bigrams = []
    for i in range(len(tags) - 1):
        tag_bigrams.append(tags[i] + " " + tags[i + 1])
    return tag_bigrams


    
def get_tags(s): 
    """Given a sequence of word/POS elements (lemmas), this function
    returns a list containing just the POS elements, in order.    
    """
    return [parse_lem(lem)[1] for lem in s.strip().split(' ') if lem]


def parse_lem(lem):
    """Helper method for parsing word/POS elements. It just splits
    on the rightmost / and returns (word, POS) as a tuple of str."""
    return lem.strip().rsplit('/', 1)  

# Call to `rel_ext.experiment`:
##### YOUR CODE HERE
featurizers = [middle_bigram_pos_tag_featurizer]
middle_bigram_pos_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.836      0.359      0.660        340       5716
author                    0.735      0.332      0.591        509       5885
capital                   0.737      0.147      0.409         95       5471
contains                  0.744      0.589      0.707       3904       9280
film_performance          0.677      0.437      0.610        766       6142
founders                  0.554      0.176      0.388        380       5756
genre                     0.688      0.194      0.456        170       5546
has_sibling               0.653      0.162      0.407        499       5875
has_spouse                0.773      0.258      0.552        594       5970
is_a                      0.585      0.173      0.396        497       5873
nationality               0.373      0.073      0.205        301       5677
parents     

In [25]:
def test_middle_bigram_pos_tag_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter['<s> VBZ'] += 5
    feature_counter = middle_bigram_pos_tag_featurizer(kbt, corpus, feature_counter)
    expected = defaultdict(
        int, {'<s> VBZ':6,'VBZ DT':1,'DT JJ':1,'JJ VBN':1,'VBN IN':1,'IN </s>':1})
    assert feature_counter == expected, \
        "Expected:\n{}\nGot:\n{}".format(expected, feature_counter)

In [26]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_middle_bigram_pos_tag_featurizer(corpus)

### Bag of Synsets [2 points]

The following allows you to use NLTK's WordNet API to get the synsets compatible with _dog_ as used as a noun:

```
from nltk.corpus import wordnet as wn
dog = wn.synsets('dog', pos='n')
dog
[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]
```

This question asks you to create synset-based features from the word/tag pairs in `middle_POS`.

__To submit:__

1. A feature function `synset_featurizer` that is just like `simple_bag_of_words_featurizer` except that it returns a list of synsets derived from `middle_POS`. Stringify these objects with `str` so that they can be `dict` keys. Use `convert_tag` (included below) to convert tags to `pos` arguments usable by `wn.synsets`. The included function `test_synset_featurizer` should help verify that you've done this correctly.

2. A call to `rel_ext.experiment` with `synset_featurizer` as the only featurizer. (Aside from this, use all the default values for `rel_ext.experiment`.)

In [27]:
from nltk.corpus import wordnet as wn

def synset_featurizer(kbt, corpus, feature_counter):
    
    ##### YOUR CODE HERE
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for synset in get_synsets(ex.middle_POS):
            feature_counter[str(synset)] += 1

    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for synset in get_synsets(ex.middle_POS):
            feature_counter[str(synset)] += 1

    return feature_counter


def get_synsets(s):
    """Suggested helper method for `synset_featurizer`. This should
    be completed so that it returns a list of stringified Synsets 
    associated with elements of `s`.
    """   
    # Use `parse_lem` from the previous question to get a list of
    # (word, POS) pairs. Remember to convert the POS strings.
    wt = [parse_lem(lem) for lem in s.strip().split(' ') if lem]
    
    ##### YOUR CODE HERE
    synsets = []
    for word_tag in wt:
        synsets.extend(wn.synsets(word_tag[0], convert_tag(word_tag[1])))

    return synsets

    
    
def convert_tag(t):
    """Converts tags so that they can be used by WordNet:
    
    | Tag begins with | WordNet tag |
    |-----------------|-------------|
    | `N`             | `n`         |
    | `V`             | `v`         |
    | `J`             | `a`         |
    | `R`             | `r`         |
    | Otherwise       | `None`      |
    """        
    if t[0].lower() in {'n', 'v', 'r'}:
        return t[0].lower()
    elif t[0].lower() == 'j':
        return 'a'
    else:
        return None    


# Call to `rel_ext.experiment`:
##### YOUR CODE HERE    
featurizers = [synset_featurizer]
synset_results = rel_ext.experiment(
    splits,
    train_split='train',
    test_split='dev',
    featurizers=featurizers,
    model_factory=model_factory,
    verbose=True)



relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.830      0.329      0.636        340       5716
author                    0.762      0.446      0.667        509       5885
capital                   0.677      0.221      0.479         95       5471
contains                  0.783      0.579      0.731       3904       9280
film_performance          0.759      0.546      0.704        766       6142
founders                  0.772      0.374      0.636        380       5756
genre                     0.585      0.224      0.442        170       5546
has_sibling               0.733      0.214      0.494        499       5875
has_spouse                0.888      0.306      0.644        594       5970
is_a                      0.619      0.235      0.467        497       5873
nationality               0.495      0.150      0.338        301       5677
parents     

In [28]:
def test_synset_featurizer(corpus):
    from collections import defaultdict
    kbt = rel_ext.KBTriple(rel='worked_at', sbj='Randall_Munroe', obj='xkcd')
    feature_counter = defaultdict(int)
    # Make sure `feature_counter` is being updated, not reinitialized:
    feature_counter["Synset('be.v.01')"] += 5
    feature_counter = synset_featurizer(kbt, corpus, feature_counter)
    # The full return values for this tend to be long, so we just
    # test a few examples to avoid cluttering up this notebook.
    test_cases = {
        "Synset('be.v.01')": 6,
        "Synset('embody.v.02')": 1
    }
    for ss, expected in test_cases.items():   
        result = feature_counter[ss]
        assert result == expected, \
            "Incorrect count for {}: Expected {}; Got {}".format(ss, expected, result)

In [29]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_synset_featurizer(corpus)

### Your original system [3 points]

There are many options, and this could easily grow into a project. Here are a few ideas:

- Try out different classifier models, from `sklearn` and elsewhere.
- Add a feature that indicates the length of the middle.
- Augment the bag-of-words representation to include bigrams or trigrams (not just unigrams).
- Introduce features based on the entity mentions themselves. <!-- \[SPOILER: it helps a lot, maybe 4% in F-score. And combines nicely with the directional features.\] -->
- Experiment with features based on the context outside (rather than between) the two entity mentions — that is, the words before the first mention, or after the second.
- Try adding features which capture syntactic information, such as the dependency-path features used by Mintz et al. 2009. The [NLTK](https://www.nltk.org/) toolkit contains a variety of [parsing algorithms](http://www.nltk.org/api/nltk.parse.html) that may help.
- The bag-of-words representation does not permit generalization across word categories such as names of people, places, or companies. Can we do better using word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/)?

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.

In [48]:
# Enter your system description in this cell.

# Final system uses the following:
# a. chaining featurizers -- helps
# b. lightgbm classifier -- helps

# Other things tried out:
# 1. context words as features with flexible window size -- did not help
# 2. direction-based length features -- helps sometimes. keeping it out of final system.
# 3. different values for `train_sampling_rate` -- interestingly found the default (0.1) gives best results 
# 4. chaining featurizers -- helps
# 5. lightgbm classifier -- helps

# My peak score was: 0.674

if 'IS_GRADESCOPE_ENV' not in os.environ:
    
    import lightgbm
    
    # ex : example
    # suffix: suffix we want to prepend to word
    # feature_counter : dict we are working with
    # window_size : number of context words.
    def add_context(ex, suffix, feature_counter, window_size):
        
        # get preceeding words in left context  
        left_context = ex.left.split(' ')
        left_context_window = left_context[len(left_context)-window_size:]
        for word in left_context_window:
            feature_counter[word + suffix] += 1

        # get succeeding words in right context  
        right_context = ex.right.split(' ')
        right_context_window = right_context[:window_size]
        for word in right_context_window:
            feature_counter[word + suffix] += 1
    
    # ex : example
    # suffix: suffix we want to prepend to length
    # feature_counter : dict we are working with
    def add_feature_counts(ex, suffix, feature_counter):
        # get feature counts 
        feature_counter['###left_chars_count' + suffix] += len(ex.left)
        feature_counter['###middle_chars_count' + suffix] += len(ex.middle)
        feature_counter['###right_chars_count' + suffix] += len(ex.right)
        
        feature_counter['###left_words_count' + suffix] += len(ex.left.split(' '))
        feature_counter['###middle_words_count' + suffix] += len(ex.middle.split(' '))
        feature_counter['###right_words_count' + suffix] += len(ex.right.split(' '))        

    
    def directional_bag_of_words_featurizer_extended(kbt, corpus, feature_counter): 
        # Append these to the end of the keys you add/access in 
        # `feature_counter` to distinguish the two orders. You'll
        # need to use exactly these strings in order to pass 
        # `test_directional_bag_of_words_featurizer`.
        subject_object_suffix = "_SO"
        object_subject_suffix = "_OS"
    
        # specify window size         
        window_size = 10
        
        ##### YOUR CODE HERE
        for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
            for word in ex.middle.split(' '):
                feature_counter[word + subject_object_suffix] += 1            
            
            # add context words
            # add_context(ex, subject_object_suffix, feature_counter, window_size)
            
            # add count features
            # add_feature_counts(ex, subject_object_suffix, feature_counter)
                
        for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
            for word in ex.middle.split(' '):
                feature_counter[word + object_subject_suffix] += 1
            
            # add context words
            # add_context(ex, object_subject_suffix, feature_counter, window_size)
            
            # add count features
            # add_feature_counts(ex, object_subject_suffix, feature_counter)
            
        return feature_counter

    model_factory = lambda: lightgbm.LGBMClassifier()
    
    featurizers_list = [
        #[simple_bag_of_words_featurizer],
        #[directional_bag_of_words_featurizer_extended],
        #[simple_bag_of_words_featurizer, directional_bag_of_words_featurizer_extended],
        #[simple_bag_of_words_featurizer, directional_bag_of_words_featurizer_extended, middle_bigram_pos_tag_featurizer],
        #[simple_bag_of_words_featurizer, directional_bag_of_words_featurizer_extended, synset_featurizer],
        [simple_bag_of_words_featurizer, directional_bag_of_words_featurizer_extended, middle_bigram_pos_tag_featurizer, synset_featurizer]
    ]
       
    train_sampling_rates = [
        # 0.05,
        0.1,
        # 0.2
    ]
    
    for featurizers in featurizers_list:
        for train_sampling_rate in train_sampling_rates:
            print("{0}:{1}".format(train_sampling_rate, featurizers))
        
            train_results = rel_ext.experiment(
                splits,
                train_split='train',
                train_sampling_rate=train_sampling_rate,
                test_split='dev',
                featurizers=featurizers,
                model_factory=model_factory,
                verbose=True)

# Please do not remove this comment.



0.1:[<function simple_bag_of_words_featurizer at 0x00000158C70FEC18>, <function directional_bag_of_words_featurizer_extended at 0x0000015951E9F0D8>, <function middle_bigram_pos_tag_featurizer at 0x00000158A14413A8>, <function synset_featurizer at 0x00000158D733B9D8>]
relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.876      0.438      0.730        340       5716
author                    0.846      0.690      0.809        509       5885
capital                   0.604      0.305      0.505         95       5471
contains                  0.789      0.789      0.789       3904       9280
film_performance          0.836      0.745      0.816        766       6142
founders                  0.837      0.447      0.713        380       5756
genre                     0.714      0.412      0.623        170       5546
has_sibling               0.849      0.541      

## Bake-off [1 point]

For the bake-off, we will release a test set. The announcement will go out on the discussion forum. You will evaluate your custom model from the previous question on these new datasets using the function `rel_ext.bake_off_experiment`. Rules:

1. Only one evaluation is permitted.
1. No additional system tuning is permitted once the bake-off has started.

The cells below this one constitute your bake-off entry.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points. Similarly, you cannot win the bake-off unless your homework is submitted on time.

The announcement will include the details on where to submit your entry.

In [49]:
# Enter your bake-off assessment code in this cell. 
# Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # Please enter your code in the scope of the above conditional.
    ##### YOUR CODE HERE
    
    import lightgbm
    
    featurizers = [simple_bag_of_words_featurizer, directional_bag_of_words_featurizer_extended, middle_bigram_pos_tag_featurizer, synset_featurizer]
    model_factory = lambda: lightgbm.LGBMClassifier()
    
    bakeoff_results = rel_ext.experiment(
        splits,
        train_split='all',
        test_split='dev',
        featurizers=featurizers,
        model_factory=model_factory,
        verbose=False) # We don't care about this eval, so skip its summary.

    rel_ext_data_home_test = os.path.join(rel_ext_data_home, 'bakeoff-rel_ext-test-data')
    rel_ext.bake_off_experiment(bakeoff_results, rel_ext_data_home_test)


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
adjoins                   0.873      0.489      0.755        438       7122
author                    0.865      0.688      0.823        645       7329
capital                   0.688      0.478      0.632        115       6799
contains                  0.741      0.790      0.750       3808      10492
film_performance          0.867      0.758      0.843       1011       7695
founders                  0.828      0.498      0.731        444       7128
genre                     0.808      0.426      0.685        188       6872
has_sibling               0.904      0.576      0.811        717       7401
has_spouse                0.872      0.427      0.721        780       7464
is_a                      0.745      0.349      0.607        611       7295
nationality               0.658      0.277      0.516        383       7067
parents     

In [50]:
# On an otherwise blank line in this cell, please enter
# your macro-average f-score (an F_0.5 score) as reported 
# by the code above. Please enter only a number between 
# 0 and 1 inclusive. Please do not remove this comment.
if 'IS_GRADESCOPE_ENV' not in os.environ:
    # pass
    # Please enter your score in the scope of the above conditional.
    ##### YOUR CODE HERE
    print("0.687")




0.687
